In [ ]:
!pip install ase
!pip install mace-torch
!pip install orb_models
!pip install pynanoflann



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 61.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.7/387.7 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
 

In [ ]:
# Pour décompresser les contenus des fichiers `mols.zip` et `solids.zip`, uniquement dans Google Colab. Décompressez manuellement sur un système local.

import os

# Créer les sous-répertoires s'ils n'existent pas.
if not os.path.exists("mols"):
  os.makedirs("mols")
if not os.path.exists("solids"):
  os.makedirs("solids")

!unzip mols.zip -d mols
!unzip solids.zip -d solids

In [ ]:
import sys
import time
from ase.calculators.calculator import Calculator as ASECalculator
from ase.io import read
from ase.filters import FrechetCellFilter
from ase.optimize import QuasiNewton, FIRE
from mace.calculators import mace_mp, mace_anicc
from orb_models.forcefield import pretrained
from orb_models.forcefield.calculator import ORBCalculator


In [ ]:
def compute_x23_case(
    mol_file: str,
    solid_file: str,
    calc: ASECalculator,
    num_mols_per_cell: int,
    f_cutoff: float=0.05,
    max_steps: int=200,
    opt_method: str="qn",
) -> tuple[float, float]:
    """
    Calcule un cas X23 donné et renvoie l'énergie réticulaire + le volume de la cellule.

    :param mol_file: Chemin vers le fichier de la molécule.
    :param solid_file: Chemin vers le fichier du solide.
    :param calc: L'outil de calcul ASE.
    :param num_mols_per_cell: Nombre de molécules dans `solid_file`.
    :param f_cutoff: Seuil des forces pour la convergence, en eV/A.
    :param max_steps: Nombre maximum d'étapes d'optimisation.
    :param opt_method: Méthode d'optimisation ("qn" ou "fire").
    """
    mol_atoms = read(mol_file)
    mol_atoms.calc = calc

    if opt_method == "qn":
        mol_qn = QuasiNewton(mol_atoms)
    elif opt_method == "fire":
        mol_qn = FIRE(mol_atoms)
    else:
        raise ValueError("unknown opt_method")
    mol_qn.run(fmax=f_cutoff, steps=max_steps)

    solid_atoms = read(solid_file)
    solid_atoms.calc = calc

    if opt_method == "qn":
        solid_qn = QuasiNewton(FrechetCellFilter(solid_atoms))
    elif opt_method == "fire":
        solid_qn = FIRE(FrechetCellFilter(solid_atoms))
    else:
        raise ValueError("unknown opt_method")
    solid_qn.run(fmax=f_cutoff, steps=max_steps)

    sE = solid_atoms.get_potential_energy()
    mE = mol_atoms.get_potential_energy()
    lattice_energy = sE / num_mols_per_cell - mE

    opt_volume = solid_atoms.get_volume() / num_mols_per_cell

    return float(lattice_energy) * 23.0605419, float(opt_volume)

In [ ]:
calc = mace_mp()

num_per_cell = [2, 4, 2, 4, 2, 4, 4, 8, 4, 2, 4, 1, 4, 2, 4, 2, 2, 8, 2, 6, 6, 4, 2]

for idx, num in enumerate(num_per_cell):
    i = idx + 1
    start_time = time.time()

    E, vol = compute_x23_case(
        f"x23/mols/{i:02d}.extxyz",
        f"x23/solids/{i:02d}.extxyz",
        calc,
        num,
    )

    end_time = time.time()
    elapsed = end_time - start_time

    with open("mace_mp_x23.out", "a+") as f:
        f.write(f"{i},{E:.5f},{vol:.5f},{elapsed:.1f}\n")

Using medium MPA-0 model as default MACE-MP model, to use previous (before 3.10) default model please specify 'medium' as model argument
Using Materials Project MACE for MACECalculator with /root/.cache/mace/macempa0mediummodel
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.
Default dtype float32 does not match model dtype float64, converting models to float32.


/usr/local/lib/python3.11/dist-packages/mace/calculators/mace.py:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_location=device)


                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 16:49:35      -95.179428       0.8199
BFGSLineSearch:    1[  2] 16:49:35      -95.214027       0.9919
BFGSLineSearch:    2[  4] 16:49:35      -95.244507       0.3852
BFGSLineSearch:    3[  6] 16:49:35      -95.259109       0.2653
BFGSLineSearch:    4[  8] 16:49:35      -95.264130       0.1279
BFGSLineSearch:    5[  9] 16:49:35      -95.270645       0.2663
BFGSLineSearch:    6[ 11] 16:49:36      -95.271751       0.0776
BFGSLineSearch:    7[ 12] 16:49:36      -95.273705       0.1324
BFGSLineSearch:    8[ 14] 16:49:36      -95.277336       0.1000
BFGSLineSearch:    9[ 17] 16:49:36      -95.284531       0.4330
BFGSLineSearch:   10[ 19] 16:49:36      -95.289795       0.1106
BFGSLineSearch:   11[ 21] 16:49:36      -95.290970       0.0745
BFGSLineSearch:   12[ 23] 16:49:36      -95.292130       0.1008
BFGSLineSearch:   13[ 25] 16:49:36      -95.292572       0.0413
                Step[ FC]     Time     

In [ ]:
orbff = pretrained.orb_d3_v2()
calc = ORBCalculator(orbff)

num_per_cell = [2, 4, 2, 4, 2, 4, 4, 8, 4, 2, 4, 1, 4, 2, 4, 2, 2, 8, 2, 6, 6, 4, 2]

for idx, num in enumerate(num_per_cell):
    i = idx + 1
    start_time = time.time()

    E, vol = compute_x23_case(
        f"x23/mols/{i:02d}.extxyz",
        f"x23/solids/{i:02d}.extxyz",
        calc,
        num,
        opt_method="fire",
    )

    end_time = time.time()
    elapsed = end_time - start_time

    with open("orbd3v2_x23.out", "a+") as f:
        f.write(f"{i},{E:.5f},{vol:.5f},{elapsed:.1f}\n")

/usr/local/lib/python3.11/dist-packages/orb_models/forcefield/pretrained.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(local_path, map_locatio

      Step     Time          Energy          fmax
FIRE:    0 17:00:31      -96.653297        1.007228
FIRE:    1 17:00:31      -96.633682        0.217597
FIRE:    2 17:00:31      -96.631996        0.153707
FIRE:    3 17:00:31      -96.630096        0.074676
FIRE:    4 17:00:31      -96.631241        0.025761
      Step     Time          Energy          fmax
FIRE:    0 17:00:31     -196.291367        0.945691
FIRE:    1 17:00:31     -196.369873        0.506170
FIRE:    2 17:00:31     -196.371933        0.341869
FIRE:    3 17:00:32     -196.371338        0.112921
FIRE:    4 17:00:32     -196.360718        0.202226
FIRE:    5 17:00:32     -196.361572        0.189059
FIRE:    6 17:00:32     -196.363205        0.164266
FIRE:    7 17:00:33     -196.365372        0.131677
FIRE:    8 17:00:33     -196.367661        0.096792
FIRE:    9 17:00:33     -196.369797        0.062661
FIRE:   10 17:00:34     -196.371506        0.065596
FIRE:   11 17:00:34     -196.372574        0.085520
FIRE:   12 17:00